In [6]:
import csv
import pandas as pd
import numpy as np
import re

# 🔹 FULL PATH TO YOUR FOLDER
base_path = r"C:\Users\Vishal\Desktop\Hackthon"

orders = pd.read_csv(base_path + r"\orders.csv")
users = pd.read_json(base_path + r"\users.json")

data = []
with open(base_path + r"\restaurants.sql", "r", encoding="utf-8") as f:
    for line in f:
        if line.startswith("INSERT INTO"):
            values = re.findall(r"\((.*?)\)", line)[0]
            parts = next(csv.reader([values], skipinitialspace=True))
            data.append(parts)

restaurants = pd.DataFrame(
    data, 
    columns=['restaurant_id', 'restaurant_name', 'cuisine', 'rating']
)

restaurants['restaurant_id'] = restaurants['restaurant_id'].astype(int)
restaurants['rating'] = restaurants['rating'].astype(float)

df = orders.merge(users, on='user_id', how='left') \
           .merge(restaurants, on='restaurant_id', how='left')

df = df.rename(columns={
    'restaurant_name_x': 'restaurant_name',
    'name': 'user_name',
    'city': 'user_city'
})

df = df.drop(columns=['restaurant_name_y'], errors='ignore')

df['order_date'] = pd.to_datetime(df['order_date'], format="%d-%m-%Y")

df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,user_name,user_city,membership,cuisine,rating
0,1,2508,450,2023-02-18,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,'Mexican',3.2
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,'Indian',4.5
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,'Mexican',4.0
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,'Chinese',4.8
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,'Italian',3.0


In [22]:
df[df['membership']=='Gold'] \
  .groupby('user_city')['total_amount'] \
  .sum() \
  .sort_values(ascending=False)


user_city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [23]:
df.groupby('cuisine')['total_amount'] \
  .mean() \
  .sort_values(ascending=False)


cuisine
'Mexican'    808.021344
'Italian'    799.448578
'Indian'     798.466011
'Chinese'    798.389020
Name: total_amount, dtype: float64

In [24]:
(df.groupby('user_id')['total_amount'].sum() > 1000).sum()

np.int64(2544)

In [25]:
df['rating_range'] = pd.cut(df['rating'],
                            bins=[3,3.5,4,4.5,5],
                            labels=['3.0–3.5','3.6–4.0','4.1–4.5','4.6–5.0'])

df.groupby('rating_range', observed=False)['total_amount'].sum().sort_values(ascending=False)

rating_range
4.6–5.0    2197030.75
4.1–4.5    1960326.26
3.0–3.5    1881754.57
3.6–4.0    1717494.41
Name: total_amount, dtype: float64

In [37]:
df[df['membership']=='Gold'] \
  .groupby('user_city')['total_amount'] \
  .mean() \
  .sort_values(ascending=False)


user_city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [38]:
df.groupby('cuisine').agg({
    'restaurant_id':'nunique',
    'total_amount':'sum'
}).sort_values('restaurant_id')


,restaurant_id,total_amount
cuisine,,
'Chinese',120,1930504.65
'Indian',126,1971412.58
'Italian',126,2024203.80
'Mexican',128,2085503.09


In [39]:
options = [
    'Grand Cafe Punjabi',
    'Grand Restaurant South Indian',
    'Ruchi Mess Multicuisine',
    'Ruchi Foods Chinese'
]

df[df['restaurant_name'].isin(options)] \
  .groupby('restaurant_name')['total_amount'] \
  .agg(['count','mean']) \
  .query('count < 20') \
  .sort_values('mean', ascending=False)


,count,mean
restaurant_name,,
Ruchi Foods Chinese,19,686.603158


In [40]:
df.groupby(['membership','cuisine'])['total_amount'] \
  .sum() \
  .sort_values(ascending=False)


membership  cuisine  
Regular     'Mexican'    1072943.30
            'Italian'    1018424.75
Gold        'Mexican'    1012559.79
            'Italian'    1005779.05
Regular     'Indian'      992100.27
Gold        'Indian'      979312.31
            'Chinese'     977713.74
Regular     'Chinese'     952790.91
Name: total_amount, dtype: float64

In [41]:
df['quarter'] = df['order_date'].dt.quarter
df.groupby('quarter')['total_amount'].sum().sort_values(ascending=False)


quarter
3    2037385.10
4    2018263.66
1    2010626.64
2    1945348.72
Name: total_amount, dtype: float64

In [42]:
gold_orders = len(df[df['membership'] == 'Gold'])
gold_orders


4987

In [43]:
hyderabad_revenue = round(df[df['user_city'] == 'Hyderabad']['total_amount'].sum())
hyderabad_revenue


1889367

In [44]:
active_users = df['user_id'].nunique()
active_users


2883

In [45]:
gold_avg_order = round(df[df['membership'] == 'Gold']['total_amount'].mean(), 2)
gold_avg_order


np.float64(797.15)

In [46]:
high_rating_orders = len(df[df['rating'] >= 4.5])
high_rating_orders


3374

In [47]:
top_city = df[df['membership'] == 'Gold'] \
              .groupby('user_city')['total_amount'] \
              .sum() \
              .idxmax()
top_city
orders_top_city_gold = len(df[(df['membership'] == 'Gold') & (df['user_city'] == top_city)])
orders_top_city_gold


1337